In [31]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import re
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

pd.set_option('display.max_colwidth', -1)

### Predict Genre Based on Plot

In [3]:
omdb = pd.read_json('movies/data/omdb-data.json.gz', lines=True)

In [102]:
#X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(omdb.omdb_plot, omdb.omdb_genres, test_size=0.2)

In [82]:
# ignore this and just use below pipeline
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(omdb['omdb_plot'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, omdb['omdb_genres'])
plot = ['hi']
X_new_counts = count_vect.transform(plot)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)
predicted[0]

In [161]:
clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,1))),
    ('tfidf',TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None))
])
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.4896694214876033


In [156]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
              'clf__penalty': ['none', 'l2', 'l1']
}
gs_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(omdb.omdb_plot, omdb.omdb_genres)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [ ]:
# logs
# svc - 0.4793388429752066
# knn - 0.303202479338843

In [157]:
gs_clf.best_score_ 

0.4708557255064076

In [158]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
clf__penalty: 'l2'
tfidf__use_idf: True
vect__ngram_range: (1, 1)


In [171]:
predicted = clf.predict(['Sam Bell has a three year contract to work for Lunar Industries. For the contract\'s entire duration, he is the sole employee based at their lunar station. His primary job responsibility is to harvest and periodically rocket back to Earth supplies of helium-3, the current clean and abundant fuel used on Earth. There is no direct communication link available between the lunar station and Earth, so his only direct real-time interaction is with GERTY, the intelligent computer whose function is to attend to his day to day needs. With such little human contact and all of it indirect, he feels that three years is far too long to be so isolated; he knows he is beginning to hallucinate as the end of his three years approaches. All he wants is to return to Earth to be with his wife Tess and their infant daughter Eve, who was born just prior to his leaving for this job. With two weeks to go, he gets into an accident at one of the mechanical harvesters and is rendered unconscious. Injured, he awakens back at the station in the infirmary, he assumes assisted by GERTY. GERTY tells him that a rescue team named Eliza will come to the station to clean up the aftermath of the accident. After his recuperation, he takes an unauthorized trip back to the broken harvester, where he makes an unexpected discovery. Because of his find, he begins to doubt his sanity, then his true identity, then the company and GERTY\'s willingness to do what is best for him. Because of his resulting beliefs, his sole mission becomes how to get back to Earth on his own.'])

In [174]:
predicted

array(['Drama'], dtype='<U11')

In [61]:
X_train, X_test, y_train, y_test = train_test_split(omdb.omdb_plot, omdb.omdb_genres, test_size=0.2)
lb = MultiLabelBinarizer()
Y = lb.fit_transform(y_train.values)

In [62]:
clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,1))),
    ('tfidf',TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1))
])

In [63]:
clf.fit(X_train, Y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...lti_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1))])

In [64]:
predicted = clf.predict(['Sam Bell has a three year contract to work for Lunar Industries. For the contract\'s entire duration, he is the sole employee based at their lunar station. His primary job responsibility is to harvest and periodically rocket back to Earth supplies of helium-3, the current clean and abundant fuel used on Earth. There is no direct communication link available between the lunar station and Earth, so his only direct real-time interaction is with GERTY, the intelligent computer whose function is to attend to his day to day needs. With such little human contact and all of it indirect, he feels that three years is far too long to be so isolated; he knows he is beginning to hallucinate as the end of his three years approaches. All he wants is to return to Earth to be with his wife Tess and their infant daughter Eve, who was born just prior to his leaving for this job. With two weeks to go, he gets into an accident at one of the mechanical harvesters and is rendered unconscious. Injured, he awakens back at the station in the infirmary, he assumes assisted by GERTY. GERTY tells him that a rescue team named Eliza will come to the station to clean up the aftermath of the accident. After his recuperation, he takes an unauthorized trip back to the broken harvester, where he makes an unexpected discovery. Because of his find, he begins to doubt his sanity, then his true identity, then the company and GERTY\'s willingness to do what is best for him. Because of his resulting beliefs, his sole mission becomes how to get back to Earth on his own.'])

In [65]:
predicted[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0])

In [66]:
lb.inverse_transform(predicted[0])

IndexError: tuple index out of range

In [76]:
y_train.groupby(level=0).first()

0       [Drama, History, War]        
1       [Action, Adventure, Thriller]
2       [Comedy, Horror]             
4       [Crime, Drama, Thriller]     
5       [Crime, Drama, Sci-Fi]       
6       [Action, Adventure, Fantasy] 
7       [Crime, Drama, Thriller]     
8       [Action, Adventure, Sci-Fi]  
11      [Biography, Comedy, Drama]   
12      [Adventure, Drama, Fantasy]  
13      [Action, Adventure, Sci-Fi]  
16      [Action, Adventure, Family]  
17      [Action, Adventure, Thriller]
19      [Action, Adventure, Thriller]
20      [Action, Drama, Sci-Fi]      
21      [Action, Adventure, Horror]  
22      [Adventure, Family, Fantasy] 
23      [Drama, Horror, Sci-Fi]      
24      [Action, Drama, Mystery]     
25      [Action, Adventure, Mystery] 
26      [Action, Sci-Fi, Thriller]   
27      [Action, Adventure, Fantasy] 
28      [Crime, Drama, Thriller]     
29      [Action, Adventure, Thriller]
30      [Biography, Crime, Drama]    
31      [Drama, Mystery, Thriller]   
32      [Act